<center><font size = "10"> Week 10 - Network Stimulation<center>
<center><font size = "8">Home Work <center>

<font size = "3">Please, write your group number and the names and surnames of the group members here
<br>Group number:
<br>Group member 1:
<br>Group member 2:
<br>Group member 3:
    
<font size='3'><font color='red'>__IMPORTANT__ The code from the tutorials generates random connections between cells, if you want to compare results you have to keep the same connections (same axon section connected to same basal section) between the cells. The same happens with the location of the exponential synapses. 
Make sure to reset your connections if necessary. HINT: In this homework all of the exercises are connected, this can help you avoid repeating the implementations.

# Load, Classes and Functions

In [1]:
!nrnivmodl

/home/NX450-2023/Preparation/week11_NetworkSimulation_I
Mod files: "./vecevent.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


In [2]:
%matplotlib inline

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuron import h, gui
from itertools import permutations
from scipy.signal import find_peaks


# Load external files & initialize
h.load_file("stdrun.hoc")
h.stdinit()

--No graphics will be displayed.


0.0

In [3]:
class Cell:
    def __init__(self, filename, name, cell_type):
        self.name = name
        self.type = cell_type
        self.synapses = []
        self.build_morphology(filename)
        self.biophysics()

    def build_morphology(self, filename):
        h.load_file("stdlib.hoc")
        h.load_file("import3d.hoc")
        if filename.endswith('.asc'):
            morph_reader = h.Import3d_Neurolucida3()
        else:
            morph_reader = h.Import3d_SWC_read()
        morph_reader.input(filename)
        i3d = h.Import3d_GUI(morph_reader, 0)
        i3d.instantiate(
            self
        )  # Notice this change to be able to instantiate several cells
        # all_sections should have the same order as neurom
        self.all_sections = self.soma + self.axon + self.dend
        if hasattr(self, 'apic'):
            self.all_sections += self.apic
        print(len(self.dend), len(self.all_sections))
        self.efferent_synapses = []
        self.afferent_synapses = []

    def biophysics(self):
        for sec in h.allsec():
            sec.Ra = 100  # Axial resistance in Ohm * cm
            sec.cm = 1  # Membrane capacitance in micro Farads / cm^2
            sec.insert("pas")
            for seg in sec:
                seg.pas.g = 0.00003
                seg.pas.e = -75

        # Insert passive current in the dendrite
        for sec in self.soma:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        if hasattr(self, "apic"):
            for sec in self.apic:
                sec.insert("hh")
                for seg in sec:
                    seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                    seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        for sec in self.dend:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

        for sec in self.axon:
            sec.insert("hh")
            for seg in sec:
                seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
                seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2

In [4]:
def spike_detector(voltage, threshold, time):
    # convert Neuron recording (voltage) to list
    voltage_read = np.array(voltage)
    # peak locations
    spike_time, _ = find_peaks(voltage_read, height=threshold)
 
    return np.array(time)[spike_time]



# <font color='red'> Exercise 1: spontaneous activity

    
For this exercise we will run a simulation of spontaneous activity by using noise to model "background" inputs.
By spontaneous we mean that no sensory input or direct stimulation is provided to the region.
The "background" inputs refer to inputs from other (not modeled) brain regions which could carry all sorts of information, but we have no clue what that information might be so we just use a noise stimulus instead.

cells.csv contains the 21 neurons from last week, morphologies contains their morphologies, and synapses.csv contains the synapses between them.

Instantiate a circuit (no need to worry about the positioning or rotation of the cells for this exercise) with those 21 cells connected by the synapses.
Set the synapses going out from all cells which have a morph_class of INT (in cells.csv) to have a reversal potential of -70mV.
    
For each neuron create a unique noise stimulus as in tutorial 1. Use an 'amp' value of 0.2.
Run a simulation for 600ms and create a spike raster plot alongside a connectivity diagram (as in tutorial 2).


    
<span style="color:purple">
<font size='3'> 
    

## **Grading**: 
- 1 pt for loading and connecting cells
- 1 pt for creating noise stimulus
- 0.5 pt each for raster and connectivity visualization
    

[/3 pt]    
</span>

In [5]:
df = pd.read_csv("cells.csv")
df

,Unnamed: 0,mtype,morph_class,morphology
0,0,L1_HAC,INT,morphologies/morph0.asc
1,1,L2_TPC:B,PYR,morphologies/morph1.asc
2,2,L2_TPC:B,PYR,morphologies/morph2.asc
3,3,L23_NBC,INT,morphologies/morph3.asc
4,4,L23_SBC,INT,morphologies/morph4.asc
5,5,L3_TPC:A,PYR,morphologies/morph5.asc
6,6,L3_TPC:A,PYR,morphologies/morph6.asc
7,7,L3_TPC:A,PYR,morphologies/morph7.asc
8,8,L3_TPC:A,PYR,morphologies/morph8.asc
9,9,L3_TPC:C,PYR,morphologies/morph9.asc


In [7]:
synapses = pd.read_csv("synapses.csv")
synapses

,Unnamed: 0,@source_node,@target_node,efferent_section_id,efferent_section_pos,afferent_section_id,afferent_section_pos
0,0,0,8,64,0.428181,213,0.887304
1,1,0,8,76,0.468049,212,0.745793
2,2,0,8,109,0.249276,216,0.970667
3,3,0,8,224,0.567864,207,0.083850
4,4,0,8,367,0.174268,206,0.513085
...,...,...,...,...,...,...,...
96,96,15,16,28,0.151952,420,0.352910
97,97,15,16,28,0.959333,420,0.524575
98,98,15,16,30,0.000926,420,0.532609
99,99,15,16,159,0.995526,481,0.269421


# <font color='red'> Ex2: Evoked activity.

Connect external synapses to cells 2, 4, and 8. Continue to apply background input as before.
    
Create a single spiketrain of poisson spiking at 40Hz starting at 200ms and stoppping at 400ms. Apply this as input to all external synapses.
Simulate the response for 600ms and create a raster plot of the network's spiking activity overlaid with the input spiking.


<span style="color:purple">
<font size='3'> 
    

## **Grading**: 
- [0.5 pt for creating synapses].
- [0.5 pt for creating correct stimulus]
- [0.5 pt for visualizing raster plot with stimulus]
    
    
[/1.5] 
</span>

# <font color= 'red'>Ex 3: Model manipulation
    
In very young mice the reversal potential of GABA-A receptors is around -30 mV. 
Let us see how such a thing influences the dynamics of our network (but note that the cortex of very young animals is different in A LOT of other ways).
    
Change the reversal potential of the inhibitory synapses and run the simulation again.
How does this change the network's spontaneous and evoked responses?
Can you explain why?


<span style="color:purple">
<font size='3'> 
    

## **Grading**: 
- [0.5 pt] for correct change
- [1 pt] for explaining the effects on network activity.

[/1.5 pt]    
</span>